<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Pedro Mesquita, 21.02028-0' #@param {type:"string"}






In [2]:
import pandas as pd

path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [3]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [4]:
df.groupby('host_name').id.count().sort_values(ascending=False).head(3)

host_name
Sonder (NYC)    7
Mark            7
David           6
Name: id, dtype: int64

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [11]:
anunciantes_multiplos = df.groupby('host_id').id.count().sort_values(ascending=False)
anunciantes_multiplos = anunciantes_multiplos[ anunciantes_multiplos > 1 ].index.tolist()
preco_mais_de_um_imovel = df[df['host_id'].isin(anunciantes_multiplos)].price.mean()
preco_mais_um_imovel = df[~df['host_id'].isin(anunciantes_multiplos)].price.mean()

print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [15]:
imoveis_manhattan = df[df['neighbourhood_group']=='Manhattan']
q1 = imoveis_manhattan.price.quantile(0.25)
q3 = imoveis_manhattan.price.quantile(0.75)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr
outliers = imoveis_manhattan[ (imoveis_manhattan.price < limite_inferior) | (imoveis_manhattan.price > limite_superior) ]
perc_Manhattan = (outliers.shape[0] / imoveis_manhattan.shape[0]) * 100
print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')


Percentuais de outliers de preços, Manhattan: 7.34 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [27]:
df['name'] = df['name'].fillna('')
df['word'] = df['name'].apply(lambda x: 'beautiful' if 'beautiful' in x.lower() else
                                      'sunny' if 'sunny' in x.lower() else
                                      'spacious' if 'spacious' in x.lower() else
                                      'heart' if 'heart' in x.lower() else
                                      'others')
df.groupby('word').price.describe()[['count', 'mean']].sort_values(by='mean', ascending=False)

count        mean
word                        
heart       36.0  166.388889
others     624.0  157.886218
beautiful   41.0  137.926829
sunny       43.0  137.674419
spacious    60.0  128.516667

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [71]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_pivotado = df_taxes.pivot(index='id', columns='type', values='value')
df_final = df_pivotado.reset_index()
df_final = df_final.fillna(0)
df_final = df.merge(df_final, on='id', how='left')
df_final.groupby('neighbourhood_group')[['fines', 'taxes']].mean().round(0)

fines   taxes
neighbourhood_group               
Bronx                126.0   851.0
Brooklyn             314.0   858.0
Manhattan            315.0   889.0
Queens               323.0   804.0
Staten Island        581.0  1158.0

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [138]:
df_report = df_final.copy()[['neighbourhood_group', 'neighbourhood', 'price']]
df_report = df_report.sort_values(by=['neighbourhood_group', 'neighbourhood'])

df_report = df_final.copy()[['neighbourhood_group', 'neighbourhood', 'price']]
df_report = df_report.sort_values(by=['neighbourhood_group', 'neighbourhood'])

report_data = []
current_neighbourhood = None
line_counter = 0
page_id = 0

for group_name, group_data in df_report.groupby('neighbourhood_group'):
    avg_price = group_data['price'].mean()

    for index, row in group_data.iterrows():
        if row['neighbourhood_group'] != current_neighbourhood:
            report_data.append(pd.DataFrame([[None, None, None, None]], columns=['neighbourhood_group', 'neighbourhood', 'price', 'page_id']))
            current_neighbourhood = row['neighbourhood_group']
            line_counter += 1
            page_id += 1
            if line_counter > 31:
                line_counter = 1

        report_data.append(pd.DataFrame([[row['neighbourhood_group'], row['neighbourhood'], row['price'], page_id]], columns=['neighbourhood_group', 'neighbourhood', 'price', 'page_id']))
        line_counter += 1
        if line_counter > 31:
            line_counter = 1
            page_id += 1

    report_data.append(pd.DataFrame([[group_name, 'MEAN', avg_price, page_id]], columns=['neighbourhood_group', 'neighbourhood', 'price', 'page_id']))


    line_counter = 1



df_report = pd.concat(report_data, ignore_index=True)

df_report = df_report.dropna()

df_report['line_id'] = df_report.groupby('page_id').cumcount() + 1


sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 13606


<ipython-input-138-ec1cfe542c7b>:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_report = pd.concat(report_data, ignore_index=True)


#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7